## Helper. Draft

In [26]:
import numpy as np
import pandas as pd

from collections import OrderedDict
import yaml

## Data

In [2]:
map_data = pd.read_csv('data/SHR65_22.csv', index_col=0)
map_data.head()

,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,Incident,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
ID,,,,,,,,,,,,,,,,,,,,,
197601001AKASP00,"Juneau, AK",AKASP00,Alaska,State Troopers,Primary state LE,FBI,Yes,1976,January,1,...,American Indian or Alaskan Native,Unknown or not reported,Knife or cutting instrument,Husband,Other arguments,NaN,0,0,30180.0,Rural Alaska
197601001AL00102,"Jefferson, AL",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00104,"Jefferson, AL",AL00104,Alabama,Fairfield,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00106,"Jefferson, AL",AL00106,Alabama,Leeds,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00201,"Mobile, AL",AL00201,Alabama,Mobile,Municipal police,FBI,Yes,1976,January,1,...,White,Unknown or not reported,Strangulation - hanging,Acquaintance,Circumstances undetermined,NaN,0,0,30180.0,"Mobile, AL"


In [12]:
map_data[map_data['Solved']=='Yes'].shape[0]

614068

In [28]:
def sk_info(state='', year=0):
    solved_or_not_info = OrderedDict()
    if not state and not year:
        solved_or_not_info['all cases'] = map_data.shape[0]
        solved_or_not_info['solved cases'] = map_data[map_data['Solved']=='Yes'].shape[0]
        solved_or_not_info['not solved cases'] = solved_or_not_info['all cases'] - solved_or_not_info['solved cases']
        solved_or_not_info['ratio solved to not solved'] = solved_or_not_info['solved cases'] / solved_or_not_info['not solved cases']
    print(yaml.dump(solved_or_not_info))

sk_info()


!!python/object/apply:collections.OrderedDict
- - [all cases, 870936]
  - [solved cases, 614068]
  - [not solved cases, 256868]
  - [ratio solved to not solved, 2.3905975053334787]

